# Used Car Sale Auction Prices
By Sky Reznik, John Lackey and Kevin Abatto
2025-04-22

Import packages

In [ ]:
import pandas as pd; # type: ignore
import numpy as np; # type: ignore
from holoviews.operation import histogram;
from sklearn.preprocessing import OneHotEncoder; # type: ignore
from sklearn.model_selection import train_test_split;
from sklearn.linear_model import LinearRegression;
from sklearn.metrics import mean_squared_error, r2_score;
import os;

Import data

In [ ]:
# Load CSV into a pandas dataframe
shared_file_path = './kaggle_datasets/car_prices.csv'
# line 408,163 - "Model" field contains a comma (SE PZEV w/Connectivity, Navigation) - specify quotechar='"'
# This tells Pandas to treat anything inside double quotes as a single field, even if it contains commas.
#      solution provided by ChatGPT
df = pd.read_csv(shared_file_path, quotechar='"', on_bad_lines='skip')

Now we need at append the MSRP of these cars. By using a data set that had make, model, year, transmission, etc. we can match the values and add in the MSRP.

We have excluded trim and chosen the lowest price match to avoid over saturating the data with multiple trim levels at different prices of the same vehicle.

In [ ]:
# Load datasets
msrp = pd.read_csv('MSRP.csv')

# Standardize column names and values
msrp.rename(columns={
    'Make': 'make',
    "Model": "model",
    "Year": "year",
    "Transmission Type": "transmission"
}, inplace=True)

msrp['transmission'] = msrp['transmission'].str.lower()

# Create matching keys
df['match_key'] = (
    df['year'].astype(str).str.lower() + '_' + 
    df['make'].str.lower() + '_' + 
    df['model'].str.lower() + '_' + 
    df['transmission'].str.lower()
)

msrp['match_key'] = (
    msrp['year'].astype(str).str.lower() + '_' + 
    msrp['make'].str.lower() + '_' + 
    msrp['model'].str.lower() + '_' + 
    msrp['transmission'].str.lower()
)

# Sort by MSRP (ascending) and keep first (min) for each match_key
msrp_min_row = msrp.sort_values('MSRP').drop_duplicates('match_key', keep='first')

# Merge with original car data (keeping all car rows but adding MSRP where matched)
car_with_msrp = df.merge(
    msrp_min_row[['match_key', 'MSRP']],  # Only keep needed columns
    on='match_key',
    how='left'
)

# Drop the temporary key column
car_with_msrp = car_with_msrp.drop(columns=['match_key'])

# Save the merged DataFrame to a new CSV file
car_with_msrp.to_csv('car_prices_with_msrp.csv', index=False)

# Data Cleaning

1) Remove any row with missing data with df.dropna()
2) One-Hot encode ['transmission'] to ['automatic_trans'] 0 /1 (double check unique vals)
3) Simplify ['body'] (collapse all 85 body types to 9 types)
4) Convert ['saledate']
5) Age of car (from ['saledate'])
6) Drop negative age cars
7) Drop unessecary or redundant columns
8) Make 'condition' numeric
9) Avoid outlier by using the 95th percentile

In [ ]:
# now we can use the car_prices_with_msrp DataFrame
df = pd.read_csv('car_prices_with_msrp.csv', low_memory=False)
# Remove 'Unamed: 16' column
df.drop('Unnamed: 16', axis=1, inplace=True)

1 - Remove rows that have missing values


In [ ]:
df.dropna(inplace=True)

2 - One-hot encode the 'transmission' column


In [ ]:
print(df['transmission'].unique())
df['auto_transmission'] = np.where(df['transmission'].str.contains('automatic', case=False), 1, 0)

3 - 'One-hot encode' the 'body' column (count 85 unique values --> reduce to 8)


In [ ]:
# ['suv' 'sedan' 'convertible' 'coupe' 'wagon' 'hatchback' 'truck' 'minivan' 'van']
df['body_type'] = np.nan
df['body_type'] = np.where(df['body'].str.contains('minivan', case=False), 'minivan', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('sedan', case=False), 'sedan', df['body'])
df['body_type'] = np.where(df['body'].str.contains('wagon', case=False), 'wagon', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('coupe', case=False), 'coupe', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('koup', case=False), 'coupe', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('convertible', case=False), 'convertible', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('hatchback', case=False), 'hatchback', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains(r'\bvan\b', case=False), 'van', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('truck', case=False), 'truck', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('cab', case=False), 'truck', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('crew', case=False), 'truck', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('suv', case=False), 'suv', df['body_type'])
df['body_type'] = np.where(df['body'].str.contains('Minivan', case=False), 'minivan', df['body_type'])

# Numerical mapping for body types
size_mapping = {
    'convertible': 0,
    'coupe': 1,
    'hatchback': 2,
    'sedan': 3,
    'wagon': 4,
    'suv': 5,
    'minivan': 6,
    'truck': 7,
    'van': 8
}

df['body_size'] = df['body_type'].map(size_mapping)

4 - Converting 'saledate' to datetime (solution provided by ChatGPT)

In [ ]:
# Handle invalid or unexpected values in the 'saledate' column
# Extract just the date part (e.g., "Dec 16 2014") before conversion
df['saledate'] = pd.to_datetime(
    df['saledate'].str.extract(r'(\w{3} \d{2} \d{4})')[0], 
    format='%b %d %Y', 
    errors='coerce'
)

5 - Create a new column 'car_age'

In [ ]:
df['car_age'] = np.where(df['saledate'].notna(), df['saledate'].dt.year - df['year'], np.nan)

6 - Some car ages are negative (which is not possible) because a 2015 model year can exist in 2014 and subsequently be sold

In [ ]:
# For rows with values less than 0, drop rows
print("Number of negative car ages dropped: ", df[df['car_age'] < 0].shape[0])
df.drop(df[df['car_age'] < 0].index, inplace=True)
# Drop old columns 'transmission' & 'body'
df.drop(['transmission', 'body'], axis=1, inplace=True)

7 - Drop unessecary or redundant columns

In [ ]:
df.drop(['year', 'trim', 'vin', 'color', 'interior', 'saledate', 'body_type'], axis=1, inplace=True)

8 - Make 'condition' numeric

In [ ]:
df['condition'] = pd.to_numeric(df['condition'], errors='coerce')

9 - Use only th 95th percentile of the data to avoid outliers

In [ ]:
sns.histplot(data=df, x='sellingprice')
threshold = df['sellingprice'].quantile(0.95)
df = df[df['sellingprice'] <= threshold]
sns.histplot(data=df, x='sellingprice')

10 - Creating 'sold_above_mmr'

MMR is provided in the dataset... it is the "Mannheim Market Report", an estimation of a car's selling value, which is updated nightly, and trained on millions of auction transactions. For our auction data, we can assume it is a sellers goal to surpass the MMR in the auction.

Therefore, a relevant one-hot-encoding would be if the `sellingprice` > `mmr`. While it is good practice to not have redundant columns; ones which can be inferred from the data, a binary column `sold_above_mmr` would be highly useful for #TODO geographical visualizations and determining which sellers or states outpreform their estimated MMR.

We will create this variable below:

In [ ]:
# ensure mmr and sellingprice are numeric
df['mmr'] = pd.to_numeric(df['mmr'], errors='coerce')
df['sellingprice'] = pd.to_numeric(df['sellingprice'], errors='coerce')
df['sold_above_mmr'] = np.where(df['sellingprice'] > df['mmr'], 1, 0 )
df[['sellingprice', 'mmr', 'sold_above_mmr']].sample(4)

# Exploratory Data Analysis

First, let's learn more about our data now that is has been cleaned!

In [ ]:
print(df.columns.unique())

## DataFrame Features Overview

### 1. make
- **Description**: The manufacturer/brand of the vehicle
- **Type**: Categorical (e.g., Toyota, Ford, Honda)
- **Potential Use**: Grouping vehicles by brand for analysis

### 2. model
- **Description**: The specific model name of the vehicle
- **Type**: Categorical (e.g., Camry, F-150, Escape)
- **Potential Use**: Detailed vehicle identification when combined with make

### 3. state
- **Description**: The geographical state where the vehicle is located/registered
- **Type**: Categorical (e.g., CA, TX, NY)
- **Potential Use**: Regional price analysis or demand patterns

### 4. condition
- **Description**: The physical/mechanical condition of the vehicle
- **Type**: Numerical (0-5)
- **Potential Use**: Key factor in pricing models

### 5. odometer
- **Description**: The mileage reading showing how many miles the vehicle has traveled
- **Type**: Numerical (continuous)
- **Potential Use**: Strong predictor of vehicle value and wear

### 6. seller
- **Description**: The party selling the vehicle 
- **Type**: Categorical
- **Potential Use**: Analyzing price differences between seller types

### 7. mmr (Manheim Market Report)
- **Description**: Wholesale market valuation of the vehicle caluculated and updated every 24 hours
- **Type**: Numerical (currency USD)
- **Potential Use**: Benchmark for comparing selling prices

### 8. sellingprice
- **Description**: The actual auction sale price of the vehicle
- **Type**: Numerical (currency USD)
- **Potential Use**: Target variable for price prediction models

### 9. MSRP (Manufacturer's Suggested Retail Price)
- **Description**: The original new vehicle price recommended by manufacturer
- **Type**: Numerical (currency USD)
- **Potential Use**: Baseline for depreciation calculations

### 10. auto_transmission
- **Description**: Whether the vehicle has automatic transmission
- **Type**: Boolean (0/1)
- **Potential Use**: Analyzing price differences between transmission types

### 11. body_size
- **Description**: The size classification of the vehicle
- **Type**: Ordinal numerical (1=smallest to 8=largest)
- **Potential Use**: Market segment analysis

### 12. car_age
- **Description**: The age of the vehicle in years
- **Type**: Numerical (discrete)
- **Potential Use**: Key factor in depreciation models

Here is a look at the format of the dataframe and what a line look like:

In [ ]:
print(df.sample(1))

First, lets look at some bloxplots of our numerical features to understand the spread of our data.

In [ ]:
# TODO - is this still needed? are we doing anything by brand?
len(df.make.unique())
df.make.value_counts()

# Data Visualization
Plot 'mmr' v. 'sellingprice'


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.scatter(df['sellingprice'], df['mmr'], alpha=0.2, s=5)  
sns.regplot(x='sellingprice', y='mmr', data=df, scatter=False, color='red')  
plt.title('Selling Price vs MMR')  
plt.xlabel('Selling Price')  
plt.ylabel('MMR') 
plt.show()

In [ ]:
X = df['mmr']  # Predictor (independent variable)
y = df['sellingprice']  # Target (dependent variable)
r_squared = r2_score(y, X)
print(f"R-squared (R²) between MMR and Selling Price: {r_squared}")


# Create a DataFrame to store R² values
results = pd.DataFrame(columns=['Predictor', 'R_Squared'])
results.loc[0] = ['mmr', r2_score(df['sellingprice'], df['mmr'])]

Here we can get a baseline understanding of how accurate MMR is. With an R² of .9538, it explains 95% of 'sellingprice' variance. We will see if we can create a model better than the industry standard! 

# Using features to predict selling price with Linear Regression
List of numeric features:
1. year
2. condition
3. odometer
4. auto_transmission
5. car_age
6. MSRP

In [ ]:
df_predictor = df[['condition', 'odometer', 'auto_transmission', 'car_age', 'MSRP', 'body_size']]
df_target = df['sellingprice']
X = df_predictor
y = df_target

# Split into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=39)
# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, Y_train)

Make predictions and evaluate the model

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

r2_linreg = r2_score(Y_test, y_pred)
print(f"R^2 (coefficient of determination): {r2_linreg}")

So, unsuprisingly, our linear regression model did not preform as well as MMR did.

In [ ]:
results.loc[1] = ['Linear Regression', r2_linreg]
print(results)

Ploting predictions from linear regression model vs. actual selling price

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(Y_test, y_pred, alpha=0.5, color='blue') 
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linewidth=1.5) # Perfect prediction line
plt.xlabel('Actual Selling Price') 
plt.ylabel('Predicted Selling Price')
plt.title('Actual vs Predicted Selling Price')
plt.show()